In [151]:
import numpy as np

In [153]:
c = np.matrix('2.0; 1.0; 0.0; 0.0')
b = np.matrix('-0.5; -1.0')
A = np.matrix('-1.0 -1.0 1.0 0.0; -4.0 -1.0 0.0 1.0')

In [156]:
"""
    LP data in standard form
    
    min  c*x
    s.t. Ax = b
         x >= 0
""" 
c = np.matrix('20.0; 15.0; 12.0; 0.0; 0.0')
b = np.matrix('-1.0; -1.0')
A = np.matrix('-10.0 -5.0 -4.0 1.0 0.0; -5.0 -2.0 -10.0 0.0 1.0')

In [158]:
# Initial input for the algo
B = np.arange(c.shape[0]-b.shape[0], c.shape[0])
N = np.arange(0, c.shape[0]-b.shape[0])
print("B: ", B)
print("N: ", N)
z = np.zeros(shape=(c.shape[0], 1))
x = np.zeros(shape=(c.shape[0], 1))
α = np.zeros(shape=(c.shape[0], 1))
z[N] = c[N] - A[:,N].T*np.linalg.inv(A[:,B].T)*c[B]
print("z: ", z.T)
print("z[N] > 0: {}, so B is dual feasible.".format(all(z[N]>0)))
y = np.linalg.inv(A[:,B].T)*c[B]

B:  [3 4]
N:  [0 1 2]
z:  [[20. 15. 12.  0.  0.]]
z[N] > 0: True, so B is dual feasible.


In [159]:
iteration_counter = 1

# Main Loop
while True:
    # (1) BTRAN:
    x[B] = np.linalg.inv(A[:,B])*b
    # Appearently x[N] is set to 0 every step ...
    x[N] = 0
    print("\nIteration: {} Objective value: {} Optimum: {}".format(iteration_counter, float(c.T*x), x.T))
    print("x: ", x.T)
    # (2) PRICING:
    if all(x[B]>=0):
        print("\nFound optimal base with B={} and xB={}.".format(B, x[B].T))
        break
    else:
        # Choose minimal index for leaving variable
        #i = np.argmin(x[B])
        # Choose first index with x[B][i] < 0 for leaving variable
        i = np.where(x[B]<0)[0][0]
        print("i: ", i)
        
    # (3) FTRAN:
    e = np.zeros(shape=(len(B),1))
    e[i] = 1
    w = np.linalg.inv(A[:,B].T)*e
    print("w: ", w.T)
    α[N] = -1*A[:,N].T*w
    print("α[N]: ", α[N].T)
    
    # (4) RATIO-TEST:
    if all(α[N]<=0):
        print("\nLP is unconstrained.")
        break
    else:
        # Get j from N with minimal z[j]/α[j] where α[j]>0 to determine entering variable
        j = N[0]
        for k in N[1:]:
            if α[j]<=0:
                j = k
                continue
            elif α[k] <= 0:
                continue
            else:
                if z[k]/α[k] < z[j]/α[j]:
                    j = k
        print("j: ", j)
    γ = float(z[j]/α[j])
    print("γ: ", γ)
    
    # (5) UPDATE:
    z[N] = z[N] - γ*α[N]
    y = y - γ*w
    z[B[i]] = γ
    N[np.where(N==j)]=B[i]
    B[i] = j
    
    iteration_counter += 1
        


Iteration: 1 Objective value: 0.0 Optimum: [[ 0.  0.  0. -1. -1.]]
x:  [[ 0.  0.  0. -1. -1.]]
i:  0
w:  [[1. 0.]]
α[N]:  [[10.  5.  4.]]
j:  0
γ:  2.0

Iteration: 2 Objective value: 2.0 Optimum: [[ 0.1  0.   0.   0.  -0.5]]
x:  [[ 0.1  0.   0.   0.  -0.5]]
i:  1
w:  [[-0.5  1. ]]
α[N]:  [[ 0.5 -0.5  8. ]]
j:  2
γ:  0.5

Iteration: 3 Objective value: 2.25 Optimum: [[0.075  0.     0.0625 0.     0.    ]]
x:  [[0.075  0.     0.0625 0.     0.    ]]

Found optimal base with B=[0 2] and xB=[[0.075  0.0625]].
